# Языковые модели

* [Language Modeling by Lena Voita](https://lena-voita.github.io/nlp_course/language_modeling.html)
* [Языковая модель - wiki](https://ru.wikipedia.org/wiki/%D0%AF%D0%B7%D1%8B%D0%BA%D0%BE%D0%B2%D0%B0%D1%8F_%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D1%8C)

Языковая модель — это статистическая модель, которая используется для оценки вероятности появления последовательности языковых единиц в тексте (символов, слов, предложений). Она позволяет определить, насколько вероятно одно слово или последовательность слов после других в данном контексте.

Обычно, когда говорят про языковые модели, то подразумевают модель "Left-to-Right". Такая модель предсказывает вероятность появления токена при условии некоторой последовательности токенов $w = (w_1, w_2, ..., w_n)$. Другими словами, для $w = (w_1, w_2, ..., w_n)$ мы последовательно моделируем $p(w_1)$, $p(w_2|w_1)$, ..., $p(w_t|w_{<t}) = p(w_t|w_{t-1}, ..., w_1)$, а значит совместное распределение такой последовательности:

$$p(w_1, w_2, ..., w_n) = p(w_1) p(w_2|w_1) p(w_3|w_1 w_2) ... p(w_k|w_{<k})...p(w_t|w_{<t}) = \prod_{t = 1}^n p(w_t | w_{<t})$$

То есть вероятность появления текста - это вероятность появления токенов этого текста. Поэтому можно сказать, что языковая модель отвечает на вопрос, насколько данная фраза типична/правильна для языка.

Виды языковых моделей:
* Статистическая языковая модель
    * N-граммная модель
    * Exponential LM
    * Positional LM
    * Factored LM
    * Cache LM
    * Katz's back-off model
* Нейронная языковая модель (NLM)
    * Masked LM (BERT)

## Статистическая языковая модель

Особенности:
* По корпусу текстов вычисляются вероятности появления токенов
* [Марковское свойство](https://ru.wikipedia.org/wiki/%D0%9C%D0%B0%D1%80%D0%BA%D0%BE%D0%B2%D1%81%D0%BA%D0%BE%D0%B5_%D1%81%D0%B2%D0%BE%D0%B9%D1%81%D1%82%D0%B2%D0%BE)
* Сглаживание (additive smoothing)
* Такую модель просто обучить и легко использовать
* Среди минусов можно отметить:
    * Необходимость хранения большого количества информации
    * Предположение о независимости токенов в предложении
    * Сильная зависимость модели от корпуса, на котором она обучается: если токена нет в словаре, то модель его не распознает

### N-граммная модель

В N-граммной модели единичным токеном считается N-грамма (униграмма, биграмма, триграмма и так далее) - последовательность слов $W = w_1 w_2 ... w_N$. Делается предположение: вероятность появления очередного токена в предположении зависит от предыдущих N-1 токенов:

$$p(W) = p(w_1) p(w_2 | w_1) p(w_3 | w_1 w_2) ... p(w_N | w_1 w_2 ... w_{N-1})$$

В N-граммной модели вероятность появления токена связана с его **частотой появления $C$ (count)** в корпусе:

$$p(w_N | w_1 w_2 ... w_{N-1}) = \frac{C(w_1 w_2 ... w_{N})}{C(w_1 w_2 ... w_{N-1})}$$

В реальности же может легко произойти так, что валидная N-грамма не встретится в корпусе, тогда мы не сможем вычислить для нее $p(w_N | w_1 w_2 ... w_{N-1})$. Чтобы избежать эту проблему, делается предположение о независимости токенов (**Марковское свойство**): будем учитывать не все токены от начала последовательности, а только $k < N - 1$ предыдущих токенов. Тогда при $k = 2$ мы получим:

$$p(w_1 w_2 ... w_{N}) = p(w_1) p(w_2 | w_1) p(w_3 | w_2) ... p(w_{N} | w_{N-1})$$

Вероятность встретить N-грамму в тексте - относительная частота события. Оценка вероятности на основе относительной частоты события представляет собой оценку максимального правдоподобия (**Maximum Likelihood Estimate — MLE**), поскольку именно это значение делает наблюдаемые значения максимально правдоподобными.

$$p(w_n | w_{n-1}) = \frac{C(w_{n-1}w_n)}{\sum\limits_{w} C(w_{n-1}w)} = \frac{C(w_{n-1}w_n)}{C(w_{n-1})}$$

Какие проблемы могут возникать при такой оценке? Во-первых, знаменатель дроби $C(w_{n-1})$ может быть равен нулю. Во-вторых, применение оценки MLE приводит к завышению вероятности часто происходящих событий и недооценке вероятности редких событий. Необходимо уменьшить вероятности часто происходящих событий и повысить вероятности редких событий. Это делается при помощи операции **сглаживания (smoothing)** - перераспределения вероятностной массы частых и редких событий. Так сглаживание решает проблемы разреженности языка и ограниченности корпуса.

**Лапласовское сглаживание (additive smoothing)** предлагает прибавлять единицу к частоте каждого слова и нормализовывать.

$$p(w|d) = \frac{F(w, d) + 1}{|d| + |V|}$$

где:
* $F(w,d)$ - частота $w$ в $d$
* +1 - Лапласов фактор
* $|d|$ - длина документа $d$ (общее число слов) 
* $|V|$ - размер словая

## Нейронная языковая модель (NLM)

Особенности:
* В этом подходе для предсказания следующего слова используется нейронная сеть
* По сути, это классификация на $|V|$ классов, где $V$ - это множество всех возможных токенов
* Архитектура может быть любой: FFNN, RNN, CNN и т.д.
* Необходимо научиться получать векторное представление предыдущего контекста

Архитектура простой языковой модели:
* На вход word embedding-и входной последовательности токенов $y_1, ..., y_{t-1}$
* Эмбеддинги подаются в некоторую архитектуру нейронной сети
* На выходе из нейронной сети будет получено представление входной последовательности токенов $h_t$
* Это представление подается в линейный слой (d x |V|), что можно рассмотреть как выходной эмбеддинг $e_w$
* Выходы в softmax для получения вероятностного распределения следующего токена $y_t$:

$$p(y_t|y_{<t}) = \frac{exp(h_t^T e_{y_t})}{\sum_{w \in V} exp(h_t^T e_w)}$$

В обучении нейронная сеть максимизирует вероятность верного токена из словаря $V$. Будем использовать для обучения cross-entropy loss. При этом, фактически, мы хотим максимизировать вероятность верного токена, то есть чтобы модель присваивала $p(y_t) = 1$ и 0 всем остальным. Тогда:

$$\text{Loss}(p^*, p) = -\log⁡{(p_{y_t})} = -log⁡(p(y_t|y_{<t}))$$

Как сделать предложенную модель эффективнее? Можно использовать в качестве архитектуры 2 подхода:
* Fixed-window Neural Network
* Recurrent Neural Network (RNN, LSTM, GRU)

## Стратегии генерации выходной последовательности

Жадное декодирование: просто будем генерировать по одному токен с наибольшей вероятностью после softmax. Это работает плохо. Справиться с этой проблемой позволяет **beam search** по сгенерированным последовательностям с некоторой небольшой глубиной.

Другие параметры генерации связаны с тем, насколько разнообразный ответ может дать языковая модель.

### Температура сэмплирования

Температура сэмплирования - дополнительный параметр $\tau$, встраиваемый в softmax. Если температура высокая, то показатели экспоненты будут небольшими, следовательно распределение вероятностей будет ближе к равномерному, следовательно ответы модели будут более разнообразными (случайными). И наоборот.

$$p(y_t|y_{<t}) = \frac{exp(\frac{h_t^T e_{y_t}}{\tau})}{\sum_{w \in V} exp(\frac{h_t^T e_w}{\tau})}$$

### Топ токенов

**Top-K токенов**: K - некоторая константа, которой мы будем ограничивать вариативность токенов, которые генерирует языковая модель. Так модель не будет выбирать наименее вероятные токены для генерации.

**Top-p вероятность токенов**: p - вероятность, которой мы будем ограничивать вариативность токенов. Теперь модель будет отбирать только токены, вероятность которых больше заданного порога.

## Оценка качества языковой модели

Мы ожидаем, что хорошая языковая модель будет генерировать корректные вероятноти для "правильных" текстов (не будет им "удивляться"): у "правильных" текстов высокая вероятность, у "неправильных" - низкая.

### Перплексия

Перплексия - величина, обратная средней вероятности появления слов в тестируемом тексте. Позволяет оценить вероятность появления некоторого текста в рамках построенной модели. Перплексию также можно трактовать как среднее количество слов, которые могут следовать за некоторым фиксированным словом. Таким образом, чем больше перплексия, тем выше неоднозначность, и следовательно, хуже языковая модель.

$$PP = P_a(w_1 w_2 ... w_V)^{\frac{1}{V}} = \sqrt[V]{\prod\limits_{i=1}^V \frac{1}{P(w_i | w_1 ... w_{i-1})}}$$

где $V$ - объем тестируемого текста.